In [16]:
!git clone https://github.com/GURSEWAK13/IMGUR5K-Handwriting-Dataset.git


Cloning into 'IMGUR5K-Handwriting-Dataset'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 51 (delta 17), reused 11 (delta 11), pack-reused 27 (from 1)
Receiving objects: 100% (51/51), 11.80 MiB | 19.40 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [21]:
!python download_imgur5k.py --dataset_info_dir ./dataset_info --output_dir ./images


For IMG: lRgjZ, ref hash: c64945bd74c067f29e01f2f3b5eeff60 != cur hash: d835884373f4d6c8f24742ceabe74946
For IMG: 27n5YK6l, ref hash: 7e8fa55ab314e7455a9c210f2cf047e8 != cur hash: 9eb6a5625c977f0a9666df26a77588d6
For IMG: rFuxUcxl, ref hash: 6161a1a6a33e459453f0910349e233de != cur hash: 988e335788793019dce35fe54828625d
For IMG: 6oJoQOt, ref hash: f7927ef90f67e2bf30a1ef041c6db373 != cur hash: d835884373f4d6c8f24742ceabe74946
For IMG: SSGx7KOh, ref hash: 83fe7f84eca8606fbffbd531803a5027 != cur hash: 7774dcc6b630ea4d49f4e870acab8766
For IMG: rRTTRGX, ref hash: 3bd26dd9b6d489889043a6f60d635ed3 != cur hash: d835884373f4d6c8f24742ceabe74946
For IMG: K2SWvUj, ref hash: e7eb7c619fcde0eeb895a1153ecaacd0 != cur hash: d835884373f4d6c8f24742ceabe74946
For IMG: NeVsJy7, ref hash: 924eb5398cea242b01f43e73b1a12811 != cur hash: d835884373f4d6c8f24742ceabe74946
For IMG: 7IUPrpZ, ref hash: 3e4f912a1e9d91c35c68c0880826e680 != cur hash: d835884373f4d6c8f24742ceabe74946
For IMG: uNHtMXk, ref hash: 8d11ad65

In [22]:
!pip install torch torchvision transformers opencv-python pillow numpy requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [ ]:
import os
import cv2

# Set environment variable to suppress warnings
os.environ['PYTHONWARNINGS'] = 'ignore'

# Load images
images = []
for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
        images.append(image)


In [ ]:
# Resize images
resized_images = [cv2.resize(image, (384, 384)) for image in images]


In [ ]:
import json

# Load annotations
with open('./dataset_info/imgur5k_annotations.json', 'r') as f:
    annotations = json.load(f)


In [ ]:
# Extract text labels
labels = []
for index, annotation in annotations['index_id'].items():
    ann_ids = annotations['index_to_ann_map'][index]
    text_labels = [annotations['ann_id'][ann_id]['word'] for ann_id in ann_ids]
    labels.append(' '.join(text_labels))


In [ ]:
from transformers import TrOCRProcessor

# Load the processor
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')

# Tokenize the text labels
tokenized_labels = [processor.tokenizer(label, return_tensors="pt").input_ids for label in labels]


In [ ]:
# Combine images and labels
dataset = list(zip(normalized_images, tokenized_labels))
